In [ ]:
from flask import Flask, request, jsonify
from pymongo import MongoClient
import matplotlib.pyplot as plt
import numpy as np
import base64
from io import BytesIO

app = Flask(__name__)

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")  # Replace with your MongoDB connection string
db = client["Project"]  # Replace with your database name
business_collection = db["Business"]  # Replace with your collection name
feature_classification_collection = db["pizza_business_feature_classification"]  # Collection with sentiment analysis

@app.route('/get_cities', methods=['POST'])
def get_cities():
    state = request.json['state']
    
    cities = business_collection.distinct("city", {
        "state": state,
        "$and": [
            {"categories": {"$regex": ".*pizza.*", "$options": "i"}},
            {"name": {"$regex": ".*pizza.*", "$options": "i"}}
        ]
    })
    
    return jsonify({"cities": cities})

@app.route('/get_businesses', methods=['POST'])
def get_businesses():
    state = request.json['state']
    city = request.json['city']
    businesses = business_collection.find(
        {
            "state": state,
            "city": city,
            "$and": [
                {"categories": {"$regex": "Pizza", "$options": "i"}},
                {"name": {"$regex": "Pizza", "$options": "i"}}
            ]
        },
        {"name": 1, "_id": 0}
    )
    business_list = [business["name"] for business in businesses]
    return jsonify({"businesses": business_list})

@app.route('/get_sentiment_analysis', methods=['POST'])
def get_sentiment_analysis():
    data = request.json
    business_name = data.get('business_name')
    city = data.get('city')
    state = data.get('state')
    
    if not all([business_name, city, state]):
        return jsonify({"error": "Missing required parameters"}), 400
    
    business = business_collection.find_one({
        "name": business_name,
        "city": city,
        "state": state
    })
    
    if not business:
        return jsonify({"error": "Business not found"}), 404
    
    business_id = business["business_id"]
    
    sentiment_data = feature_classification_collection.find_one({"business_id": business_id})
    
    if not sentiment_data:
        return jsonify({"error": "Sentiment analysis data not found"}), 404
    
    feature_classification = sentiment_data["feature_classification"]
    results = {}
    
    for category, category_data in feature_classification.items():
        polarity_scores = category_data.get('polarity_scores', {})
        if polarity_scores:
            results[category] = {
                "positive_score": polarity_scores.get('positive_score', 0) * 100,
                "negative_score": polarity_scores.get('negative_score', 0) * 100
            }
    
    fig, ax = plt.subplots(figsize=(10, 6))
    categories = list(results.keys())
    pos_scores = [results[cat]['positive_score'] for cat in categories]
    neg_scores = [results[cat]['negative_score'] for cat in categories]
    
    x = range(len(categories))
    width = 0.25  # Decreased bar width
    
    pos_bars = ax.bar([i - width/2 for i in x], pos_scores, width, label='Positive', color='green')
    neg_bars = ax.bar([i + width/2 for i in x], neg_scores, width, label='Negative', color='red')
    
    ax.set_ylabel('Sentiment Score (%)')
    ax.set_title(f'Sentiment Analysis for {business_name}')
    ax.set_xticks(x)
    ax.set_xticklabels(categories)
    ax.legend()
    
    # Set y-axis limit to 100%
    ax.set_ylim(0, 100)
    
    # Add percentage labels on top of bars
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}%', ha='center', va='bottom')
    
    add_labels(pos_bars)
    add_labels(neg_bars)
    
    img_io = BytesIO()
    plt.savefig(img_io, format='png', bbox_inches='tight')
    img_io.seek(0)
    img_base64 = base64.b64encode(img_io.getvalue()).decode('utf-8')
    plt.close(fig)
    
    return jsonify({"image": img_base64, "message": "Sentiment analysis completed successfully"})

@app.route('/')
def index():
    state_codes = [
        'AB', 'AZ', 'CA', 'CO', 'DE',
        'FL', 'HI', 'ID', 'IL', 'IN',
        'LA', 'MA', 'MI', 'MO', 'MT',
        'NC', 'NJ', 'NV', 'PA', 'SD',
        'TN', 'TX', 'UT', 'VI', 'VT',
        'WA', 'XMS'
    ]
    return render_template('index.html', state_codes=state_codes)

if __name__ == '__main__':
    app.run(debug=False, port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [03/Dec/2024 14:49:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 14:50:00] "POST /get_cities HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 14:50:02] "POST /get_businesses HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 14:50:05] "POST /get_sentiment_analysis HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 14:50:07] "POST /get_businesses HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 14:50:09] "POST /get_sentiment_analysis HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 14:50:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 15:47:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 15:47:59] "POST /get_cities HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 15:48:01] "POST /get_businesses HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 15:48:06] "POST /get_sentiment_analysis HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 15:48:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 16:22:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2024 